In [ ]:
!pip install python-polylabel

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import tifffile
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from PIL import Image
import cv2
import json
import struct
from polylabel import polylabel

%matplotlib inline

In [ ]:
fx = 0.1  # Percent used to resize to thumbnail

In [ ]:
input_path = Path('../input/hubmap-kidney-segmentation/')

In [ ]:
images_path = list(input_path.rglob('*.tiff'))
len(images_path)

In [ ]:
def decode_color(color):
    return list(map(ord, struct.unpack('cccc', struct.pack('i', color))))[:3][::-1]


def draw_contors(im, fx, annotations, color, thickness):
    for annotation in annotations:
        # Get coordinates
        coords = annotation['geometry']['coordinates']

        # Add a level when geometry is Polygon
        if annotation['geometry']['type'] == 'Polygon':
            coords = [coords]

        # Apply fx, convert to int and transpose to cv2 format
        coords = [np.int32(np.array(c) * fx).transpose(1, 0, 2) for c in coords]

        # Draw contours
        cv2.drawContours(im, coords, -1, color, thickness, lineType=cv2.LINE_AA)
        
        # Text to plot
        text = annotation['properties']['classification']['name']
        
        # Ignore glomerulus
        if text != 'glomerulus':
            # Font face
            font = cv2.FONT_HERSHEY_SIMPLEX

            # Contour area and sqrt(sqrt) of it. Used to find font size
            total_area = sum(map(cv2.contourArea, coords))
            sqrt_area = np.sqrt(np.sqrt(total_area))

            # Font scale, proportional to area of countour
            fontScale = 0.5 * sqrt_area / 6 * fx / 0.1

            # Find center within polygon (https://blog.mapbox.com/a-new-algorithm-for-finding-a-visual-center-of-a-polygon-7c77e6492fbc)
            cX, cY = polylabel(coords[0].transpose(1, 0, 2))

            # Text size, used to center in cX, cY
            textsize = cv2.getTextSize(text, font, fontScale, 2)[0]

            # Draw text
            cv2.putText(im, text, (int(cX-textsize[0]/2), int(cY+textsize[1]/2)), font, fontScale, color, 2)
        

def read_annotations(fname):
    if fname.exists():
        with open(fname) as f:
            return json.load(f)
    else:
        # File not found. E.g., test annotation
        return None

In [ ]:
for fname in tqdm(images_path):
    # Read image
    im = tifffile.imread(fname)
    print(fname, im.shape)
    
    if im.ndim == 5:
        # Fiz crazy shape (1, 1, c, h, w)!
        im = np.rollaxis(im[0, 0], 0, 3)

    # Convert to BGR format to be used in cv2
    im = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)
    
    # Resize to fx
    im = cv2.resize(im, None, fx=fx, fy=fx, interpolation=cv2.INTER_AREA)
    
    # Read annotations
    anatomical_data = read_annotations(fname.parent / f'{fname.stem}-anatomical-structure.json')
    annotation_data = read_annotations(fname.parent / f'{fname.stem}.json')
    
    # Draw annotations (color in BGR format)
    if annotation_data is not None:
        draw_contors(im, fx, annotation_data, [0, 255, 0], 1)

    draw_contors(im, fx, anatomical_data, [0, 0, 255], 2)
    
    # Write output file
    fname_jpeg = f'{fname.parent.name}_{fname.stem}.jpg'
    cv2.imwrite(str(fname_jpeg), im, [int(cv2.IMWRITE_JPEG_QUALITY), 95])